# Détection dynamique des lignes électriques aériennes pour le ProjetAmpere

Reprend en grande partie le code du notebook PowerLine.ipynb mais sur des rafales d'images disponibles dans le dossier. 

In [1]:
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
os.chdir(r"C:\Users\VArri\Documents\PowerLines\images\visuel")
path = os.getcwd()
print(path)

C:\Users\VArri\Documents\PowerLines\images\visuel


In [3]:
new_model = tf.keras.models.load_model('CNN/model')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 12)          1812      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 12)          0         
_________________________________________________________________
flatten (Flatten)            (None, 192)               0         
_________________________________________________________________
dense (Dense)                (None, 192)               37056     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [4]:
steered_path = os.path.join(path, 'steered1000')
print(steered_path)

C:\Users\VArri\Documents\PowerLines\images\visuel\steered1000


In [5]:
img_lst = [img_name for img_name in os.listdir(steered_path) if img_name.startswith('20210507_133045')]
# result

In [6]:
for img_path in img_lst:
    
    img_path = os.path.join(steered_path, img_path)
    img = cv2.imread(img_path, 0)
    img = img.astype('float32') /255
    
    img_shape = img.shape
    detect = np.zeros(img_shape)
    HT = int(28/2)
    
    cropped = []
    for i in tqdm(range(HT, int((img.shape[0]-HT+1)))):
        for j in range(HT, img.shape[1]-HT+1):
            cropped.append(img[i-HT:i+HT, j-HT:j+HT])

    cropped = np.array(cropped).astype(np.float32).reshape([len(cropped), 28, 28, 1])
    detect = new_model.predict(cropped).round()
    detect = detect.reshape(int((img.shape[0]-HT+1)) - HT, img.shape[1]-HT+1 - HT).astype(np.uint8)
    
    lines = cv2.HoughLinesP(image=detect, rho=1, theta=np.pi/180, threshold=50, lines=np.array([]),minLineLength=300, maxLineGap=50)
    lst = img_name.split('_')
    origin = '_'.join(lst[:-2]) + '_edge.jpg'
    # os.chdir('../edge1000')

    img = cv2.imread(os.path.join(path, 'edge1000',origin), 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    a,b,c = lines.shape
    print(lines.shape)
    for i in range(a):
        l = lines[i][0]
        cv2.line(img, (l[0]+HT, l[1]+HT), (l[2]+HT, l[3]+HT), (255,0,0), 1, cv2.LINE_AA)
        # cv2.imwrite('houghlines4.jpg', img)
    plt.figure(figsize=(16, 8)) 
    plt.imshow(img)
    plt.show()
    
    cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(path,'dyn','_'.join(lst[:-2]) + '_dyn.jpg'), img)
    

100%|██████████████████████████████████████████████████████████████████████████████| 989/989 [00:00<00:00, 1810.89it/s]


NameError: name 'img_name' is not defined